In [1]:
# TD Ameritrade OAuth 2.0 script
# This python script opens a browser,
# invokes a TD Ameritrade OAuth server
# with a url to receive an authorization code.
# Submits TD Ameritrade login information to receive tokens.
# OAuth.ipynb
# 06/21/2020
# Author Sam Portillo

# References:
#   https://oauth.net/2/
#   https://developer.tdameritrade.com/


# conda activate TDA
# conda install -n myenv scipy
# conda install -n myenv pip
# conda activate myenv
# pip <pip_subcommand>
# ipython kernel install --name TDA --user
# or
# python -m ipykernel install --user


import requests, time, urllib
from splinter import Browser
from selenium import webdriver
from credentials import client_id, username, password, redirect_uri

In [2]:
# Chrome, Help, About Google Chrome → version needs to match driver.
executable_path = {'executable_path': r'C:\Z\drivers\83\chromedriver.exe' }
browser = Browser('chrome', **executable_path, headless = False )

In [3]:
# Build OAuth 2.0 url
method = 'GET'
url = 'https://auth.tdameritrade.com/auth?'
payload = {'response_type':'code', 'redirect_uri':redirect_uri, 'client_id':client_id + '@AMER.OAUTHAP'}
oauth_url = requests.Request(method, url, params = payload).prepare()
browser.visit(oauth_url.url)

In [4]:
# Fill out each element in the form
browser.find_by_id("username").first.fill(username)
browser.find_by_id("password").first.fill(password)
browser.find_by_id("accept").first.click()

#time.sleep(3)
#browser.find_by_id("accept_pre").first.click()

# Need time to manually enter TDA text key

In [5]:
time.sleep(30)

# Click to 'Continue'
browser.find_by_id("accept").first.click()
time.sleep(1)
authorization_code = urllib.parse.unquote(browser.url.split('code=')[1])
browser.quit()

In [6]:
# Request Post Access Token
url = r'https://api.tdameritrade.com/v1/oauth2/token'
headers = {'Content-Type':"application/x-www-form-urlencoded"}
payload = {
            'grant_type':'authorization_code',
            'access_type':'offline',
            'code':authorization_code,
            'client_id':client_id,
            'redirect_uri':redirect_uri
          }

response = requests.post(url, headers = headers, data = payload)

# Convert python object into a JSON string
tokens = response.json()
#print(tokens)

In [7]:
print(tokens['access_token'])

O33bd2JkGD2cbQELgxq/mlsH4Uwhg4RYWd8c4LaXLUWFmCRTR7Ykn6b5iCgxX0IA/bsxIeXGnW28XohZ1gONfQKwhLytvu4iwyh40OOirj1/TN4ciKEAWDkNiEJHowIspJh2yM2dXiF+42k9R+oWljbBB9XNOkArXbWtSgzUjKDSx4uL1r6uBKrVNrxjQVth07RbLgLTuAxlN4eGtfBL8xn/9kd+E6mXcQyx+I3yDhET9B9aE/RWvJraKVYEUTmDtIP19O8Hm0ltFsL+VbxzEugAv+vFxWyWAzqgxA1DkvIiQCpxbArgyfzhNwqvuuQp5qwDt89vATJk0bC+RZWhecOA1QI99jdy0K8VSzVM2XfShZDHonzimCeunxzRy2+GV2NHOcKpDf1DLBQsljtBu6HIUnaJ+TCSaBjQePiFG1UNJ3aefUbF1hwl/KOgTtd+05Q2dKOPU6e2ftdY91kQ0SZhjLAJyK9UbjxxDly+QwXoG9Zmgxz+fNmslt4kbfObIurWppLVVn2Ehtav2pzHqPGthOBQh7jPuR9MYJB/Ohsd100MQuG4LYrgoVi/JHHvlu8St/Z9oP8mMMZeI0wSs1gmBxygv9BWovAj3JY9EeHgN4uNak3sDDxvlxLlbXW08O0oOezxPGYKrXFESMlF/mL4pvxeUKgnf81oZoiS1Wjx7ockAMqDpmMFboxHq+XZxBzzTs8SxG50hwAJT+5me3NVKyFGltLj6d3+85haSNuzTBIi32SNx1GwqM1bzZv6NsBqRI+YLG0TtPXKrBEZjaMky0n6bz37pQrMBiLd7tsUZpGKc5V0H5W11u00aqpUlSrtlVhzXpNkmfbruPFuqk7fzXwfesJrvsQ0lPZwl4zJXQOuCmLDtS+RkKWYV+jfeHb9Qtd+XSTXntOAm12igpWe+oWT7rLPZ9uy6MAZRQrA3E7CpPsUFE9vK/mDe2J1nDveqDhqNMntYHb7SoNjyB6/N9mnmKvv5bb8XKeV